This notebook serves as the source of all facts and figures references in my to-be-named working paper

In [8]:
from taxcalc import *
from taxcalc.utils import *
import pandas as pd
import numpy as np
from bokeh.io import show, output_notebook, export_svgs
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter
from scipy import integrate
from notebookfunctions import index_list
pd.options.display.float_format = '{:8,.2f}'.format
output_notebook()

Loading BokehJS ...

This paper examines the relationship between projected income inequality and tax revenue projections

In [9]:
def gini(pct_val, pct_wt):
    """
    Calculate the GINI coefficient for a given wage distribution
    """
    x = integrate.simps(pct_val, pct_wt)
    y = integrate.simps([0, 1], [0, 1])
    gini = (y - x) / y
    return gini

In [10]:
def lorenz(var, wt):
    """
    Return a dataframe suitable for creating a lorenz curve
    
    Parameters
    ----------
    var: variable to sort by
    wt: weight for each record
    """
    df = pd.DataFrame({'var': var, 's006': wt})
    df['var_w'] = df['var'] * df['s006']  # weight the variable
    df = add_quantile_bins(df, income_measure='var', num_bins=100)
    # group by income bin
    gdf = df.groupby('bins')
    # find the weighted sum of the variable and total weight by bin
    sdf = gdf[['var_w', 's006']].sum()
    # find the percentiles in each bin
    pdf = pd.DataFrame({'pct_var': sdf['var_w'] / sdf['var_w'].sum(),
                        'pct_s006': sdf['s006'] / sdf['s006'].sum()})
    pdf['cs_var'] = pdf['pct_var'].cumsum()
    pdf['cs_s006'] = pdf['pct_s006'].cumsum()
    return pdf[['cs_var', 'cs_s006']]

# Graphs and Data

In [11]:
fred_gini = pd.read_csv('../Desktop/Income Distribution Sensitivity/gini_fred.csv')

In [12]:
fred_gini['year'] = [year for year in range(1967, 2017)]
fred_cds = ColumnDataSource(fred_gini)

In [13]:
f = figure(title='GINI Ratio - Households', width=750)
f.line(x='year', y='GINIALLRH', source=fred_cds)
hover = HoverTool(tooltips=[('Year', '@year'), ('GINI', '@GINIALLRH')])
f.add_tools(hover)
f.xaxis.axis_label = 'Year'
f.yaxis.axis_label = 'Ratio'
f.xaxis[0].ticker.desired_num_ticks = 10
#f.toolbar_location = None
#f.toolbar.logo = None
#f.output_backend = "svg"
#export_svgs(f, '../Desktop/Income Distribution Sensitivity/plots/fred_gini.svg')
show(f)

Source: [U.S. Bureau of the Census](https://fred.stlouisfed.org/series/GINIALLRH)

### Comparison of the distribution of wages and salaries

In [14]:
dist_data = pd.read_csv('../taxdata/puf_stage3/multiyear_was_targets.csv', index_col=0)
dist_data['2000 pct'] = dist_data['2000'] / dist_data['2000'].sum()
dist_data['2004 pct'] = dist_data['2004'] / dist_data['2004'].sum()
dist_data['2009 pct'] = dist_data['2009'] / dist_data['2009'].sum()
dist_data['2014 pct'] = dist_data['2014'] / dist_data['2014'].sum()

In [15]:
dist_cds_00 = ColumnDataSource({'total': dist_data['2000'].values / 1000000,
                                'pct': dist_data['2000 pct'].values,
                                'index': index_list(),
                                'year': ['2000' for item in index_list()]})

dist_cds_04 = ColumnDataSource({'total': dist_data['2004'].values / 1000000,
                                'pct': dist_data['2004 pct'].values,
                                'index': index_list(),
                                'year': ['2004' for item in index_list()]})

dist_cds_09 = ColumnDataSource({'total': dist_data['2009'].values / 1000000,
                                'pct': dist_data['2009 pct'].values,
                                'index': index_list(),
                                'year': ['2009' for item in index_list()]})

dist_cds_14 = ColumnDataSource({'total': dist_data['2014'].values / 1000000,
                                'pct': dist_data['2014 pct'].values,
                                'index': index_list(),
                                'year': ['2014' for item in index_list()]})

In [16]:
# plot distributions
f = figure(title='Total Wages and Salaries by AGI Bin ($b)', x_range=index_list(), width=750)
f.circle(x='index', y='total', legend='2000', size=8, fill_alpha=0.5, source=dist_cds_00)
f.diamond(x='index', y='total', legend='2004', size=8, fill_alpha=0.5, color='red', source=dist_cds_04)
f.square(x='index', y='total', legend='2009', size=8, fill_alpha=0.5, color='green', source=dist_cds_09)
f.triangle(x='index', y='total', legend='2014', size=8, fill_alpha=0.5, color='orange', source=dist_cds_14)
f.xaxis.major_label_orientation = .75
hover = HoverTool(tooltips=[('Year', '@year'), ('Total', '$@total{0.2f}'), ('Bin', '@index')])
f.add_tools(hover)
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00')
f.toolbar.logo = None
#f.toolbar_location = None
#export_png(f, '../Desktop/Income Distribution Sensitivity/plots/total_was_dist.png')
show(f)

In [17]:
f = figure(title='Percent of Total Wages and Salaries by AGI Bin', x_range=index_list(), width=750)
f.circle(x='index', y='pct', legend='2000', size=8, fill_alpha=0.5, source=dist_cds_00)
f.diamond(x='index', y='pct', legend='2004', size=8, fill_alpha=0.5, color='red', source=dist_cds_04)
f.square(x='index', y='pct', legend='2009', size=8, fill_alpha=0.5, color='green', source=dist_cds_09)
f.triangle(x='index', y='pct', legend='2014', size=8, fill_alpha=0.5, color='orange', source=dist_cds_14)
f.xaxis.major_label_orientation = .75
hover = HoverTool(tooltips=[('Year', '@year'), ('Pct', '@pct{"0.00%}'), ('Bin', '@index')])
f.add_tools(hover)
f.yaxis[0].formatter = NumeralTickFormatter(format='0.00%')
show(f)

In [18]:
# baseline calculator
wt = pd.read_csv('../taxdata/puf_stage2/puf_weights.csv')
puf = pd.read_csv('../taxdata/puf_data/puf.csv')
adj = pd.read_csv('../taxdata/puf_stage3/puf_ratios.csv', index_col=0)
adj = adj.transpose()
growf = Growfactors('../taxdata/stage1/growfactors.csv')
recs = Records(puf, gfactors=growf, weights=wt, adjust_ratios=adj, start_year=2011)
pol = Policy(gfactors=growf, start_year=2011, num_years=16)

calc = Calculator(records=recs, policy=pol,
                  consumption=Consumption(start_year=2011, num_years=16),
                  behavior=Behavior(start_year=2011, num_years=16))
calc.advance_to_year(2017)
calc.calc_all()

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


In [19]:
# Calculate distribution info
was_2014 = lorenz(calc.records.e00200, calc.records.s006)
gini_2014 = gini(was_2014['cs_var'], was_2014['cs_s006'])
gini_2014

0.69084596962088285

In [20]:
# data for plotting
lorenz_14 = ColumnDataSource({'s006': was_2014['cs_s006'],
                              'var': was_2014['cs_var']})

In [21]:
# tax liability information
iitax = (calc.records.iitax * calc.records.s006).sum()
payroll = (calc.records.payrolltax * calc.records.s006).sum()
combined = (calc.records.combined * calc.records.s006).sum()

### Distribution Year: 2000

In [22]:
# calculator with 2000 distribution
adj_2000 = pd.read_csv('../taxdata/puf_stage3/puf_ratios2000.csv', index_col=0).transpose()
growf_2000 = Growfactors('../taxdata/stage1/growfactors.csv')
recs_2000 = Records(puf, gfactors=growf_2000, weights=wt, adjust_ratios=adj_2000, start_year=2011)
pol_2000 = Policy(gfactors=growf_2000, start_year=2011, num_years=16)

calc_2000 = Calculator(records=recs_2000, policy=pol_2000,
                  consumption=Consumption(start_year=2011, num_years=16),
                  behavior=Behavior(start_year=2011, num_years=16))
calc_2000.advance_to_year(2017)
calc_2000.calc_all()

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


In [23]:
# get data for lorenz curve
was_2000 = lorenz(calc_2000.records.e00200, calc_2000.records.s006)

In [24]:
gini_2000 = gini(was_2000['cs_var'], was_2000['cs_s006'])
gini_2000

0.6062897999463841

In [25]:
# prep data for plotting
lorenz_00 = ColumnDataSource({'s006': was_2000['cs_s006'],
                              'var': was_2000['cs_var']})

In [26]:
f = figure(plot_width=700, plot_height=400)
f.line('s006', 'var',
       line_width=2, legend='2014', source=lorenz_14)

f.line('s006', 'var',
       line_width=2, legend='2000', color='red', source=lorenz_00)

f.line([0, 1], [0, 1], color='black', line_width=2, line_alpha=.5)
f.yaxis.axis_label = 'Pct. of Total WAS'
f.xaxis.axis_label = 'Pct. of Total Population'
f.legend.location = 'top_left'
hover = HoverTool(tooltips=[('Pct. WAS', '@var{0.00%}'), ('Pct. Pop.', '@s006{0.00%}')])
f.add_tools(hover)
f.xaxis[0].formatter = NumeralTickFormatter(format='0.00%')
f.yaxis[0].formatter = NumeralTickFormatter(format='0.00%')

show(f)

In [27]:
# tax liability information
iitax_00 = (calc_2000.records.iitax * calc_2000.records.s006).sum()
payroll_00 = (calc_2000.records.payrolltax * calc_2000.records.s006).sum()
combined_00 = (calc_2000.records.combined * calc_2000.records.s006).sum()

In [28]:
# Show the difference in total tax liability
tax_diffs_00 = pd.DataFrame()
tax_diffs_00['Default'] = [iitax, payroll, combined]
tax_diffs_00['2000 Dist.'] = [iitax_00, payroll_00, combined_00]
tax_diffs_00['Pct. Change'] = ((tax_diffs_00['2000 Dist.'] - tax_diffs_00['Default']) /
                               tax_diffs_00['Default']) * 100
tax_diffs_00.index = ['Income Tax', 'Payroll Tax', 'Combined']
tax_diffs_00

,Default,2000 Dist.,Pct. Change
Income Tax,"1,548,171,817,587.70","1,420,366,189,571.65",-8.26
Payroll Tax,"1,104,042,336,022.68","1,100,885,075,818.81",-0.29
Combined,"2,652,214,153,610.37","2,521,251,265,390.48",-4.94


In [29]:
# % change in GINI from 2014 distribution to 2000 distribution
gini_change_00 = ((gini_2000 - gini_2014) / gini_2014) * 100
gini_change_00

-12.239511177998308

In [30]:
# Calculate elasticity for 2000
elast_00 = tax_diffs_00['Pct. Change']['Combined'] / gini_change_00
elast_00

0.40343686714039889

### Distribution Year: 2004

In [31]:
# Calculator with 2004 distribution
adj_2004 = pd.read_csv('../taxdata/puf_stage3/puf_ratios2004.csv', index_col=0).transpose()
growf_2004 = Growfactors('../taxdata/stage1/growfactors.csv')
recs_2004 = Records(puf, gfactors=growf_2004, weights=wt, adjust_ratios=adj_2004, start_year=2011)
pol_2004 = Policy(gfactors=growf_2004, start_year=2011, num_years=16)

calc_2004 = Calculator(records=recs_2004, policy=pol_2004,
                  consumption=Consumption(start_year=2011, num_years=16),
                  behavior=Behavior(start_year=2011, num_years=16))
calc_2004.advance_to_year(2017)
calc_2004.calc_all()

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


In [32]:
# get distribution data
was_2004 = lorenz(calc_2004.records.e00200, calc_2004.records.s006)
gini_2004 = gini(was_2004['cs_var'], was_2004['cs_s006'])
gini_2004

0.62173949335941814

In [33]:
# prep data for plotting
lorenz_04 = ColumnDataSource({'s006': was_2004['cs_s006'],
                              'var': was_2004['cs_var']})

In [34]:
f = figure(plot_width=700, plot_height=400)
f.line('s006', 'var',
       line_width=2, legend='2014', source=lorenz_14)

f.line('s006', 'var',
       line_width=2, legend='2009', color='red', source=lorenz_04)

f.line([0, 1], [0, 1], color='black', line_width=2, line_alpha=.5)
f.yaxis.axis_label = 'Pct. of Total WAS'
f.xaxis.axis_label = 'Pct. of Total Population'
f.legend.location = 'top_left'
f.add_tools(hover)
f.xaxis[0].formatter = NumeralTickFormatter(format='0.00%')
f.yaxis[0].formatter = NumeralTickFormatter(format='0.00%')
show(f)

In [35]:
# tax liability information
iitax_04 = (calc_2004.records.iitax * calc_2004.records.s006).sum()
payroll_04 = (calc_2004.records.payrolltax * calc_2004.records.s006).sum()
combined_04 = (calc_2004.records.combined * calc_2004.records.s006).sum()

In [36]:
# Show the difference in total tax liability
tax_diffs_04 = pd.DataFrame()
tax_diffs_04['Default'] = [iitax, payroll, combined]
tax_diffs_04['2004 Dist.'] = [iitax_04, payroll_04, combined_04]
tax_diffs_04['Pct. Change'] = ((tax_diffs_04['2004 Dist.'] - tax_diffs_04['Default']) /
                               tax_diffs_04['Default']) * 100
tax_diffs_04.index = ['Income Tax', 'Payroll Tax', 'Combined']
tax_diffs_04

,Default,2004 Dist.,Pct. Change
Income Tax,"1,548,171,817,587.70","1,416,437,619,028.48",-8.51
Payroll Tax,"1,104,042,336,022.68","1,099,982,026,238.76",-0.37
Combined,"2,652,214,153,610.37","2,516,419,645,267.22",-5.12


In [37]:
gini_change_04 = ((gini_2004 - gini_2014) / gini_2014) * 100
gini_change_04

-10.003167030038322

In [38]:
elast_04 = tax_diffs_04['Pct. Change']['Combined'] / gini_change_04
elast_04

0.51184221019873988

### Distribution Year: 2009

In [39]:
# Calculator with 2009 distribution
adj_2009 = pd.read_csv('../taxdata/puf_stage3/puf_ratios2009.csv', index_col=0).transpose()
growf_2009 = Growfactors('../taxdata/stage1/growfactors.csv')
recs_2009 = Records(puf, gfactors=growf_2009, weights=wt, adjust_ratios=adj_2009, start_year=2011)
pol_2009 = Policy(gfactors=growf_2009, start_year=2011, num_years=16)

calc_2009 = Calculator(records=recs_2009, policy=pol_2009,
                  consumption=Consumption(start_year=2011, num_years=16),
                  behavior=Behavior(start_year=2011, num_years=16))
calc_2009.advance_to_year(2017)
calc_2009.calc_all()

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


In [40]:
# get distribution data
was_2009 = lorenz(calc_2009.records.e00200, calc_2009.records.s006)
gini_2009 = gini(was_2009['cs_var'], was_2009['cs_s006'])
gini_2009

0.65433218879243005

In [41]:
# prep data for plotting
lorenz_09 = ColumnDataSource({'s006': was_2004['cs_s006'],
                              'var': was_2004['cs_var']})

In [42]:
f = figure(plot_width=700, plot_height=400)
f.line('s006', 'var',
       line_width=2, legend='2014', source=lorenz_14)

f.line('s006', 'var',
       line_width=2, legend='2009', color='red', source=lorenz_09)

f.line([0, 1], [0, 1], color='black', line_width=2, line_alpha=.5)
f.yaxis.axis_label = 'Pct. of Total WAS'
f.xaxis.axis_label = 'Pct. of Total Population'
f.legend.location = 'top_left'
f.add_tools(hover)
f.xaxis[0].formatter = NumeralTickFormatter(format='0.00%')
f.yaxis[0].formatter = NumeralTickFormatter(format='0.00%')
show(f)

In [43]:
# tax liability information
iitax_09 = (calc_2009.records.iitax * calc_2009.records.s006).sum()
payroll_09 = (calc_2009.records.payrolltax * calc_2009.records.s006).sum()
combined_09 = (calc_2009.records.combined * calc_2009.records.s006).sum()

In [44]:
# Show the difference in total tax liability
tax_diffs_09 = pd.DataFrame()
tax_diffs_09['Default'] = [iitax, payroll, combined]
tax_diffs_09['2004 Dist.'] = [iitax_09, payroll_09, combined_09]
tax_diffs_09['Pct. Change'] = ((tax_diffs_09['2004 Dist.'] - tax_diffs_09['Default']) /
                               tax_diffs_09['Default']) * 100
tax_diffs_09.index = ['Income Tax', 'Payroll Tax', 'Combined']
tax_diffs_09

,Default,2004 Dist.,Pct. Change
Income Tax,"1,548,171,817,587.70","1,459,321,467,413.01",-5.74
Payroll Tax,"1,104,042,336,022.68","1,100,588,943,655.76",-0.31
Combined,"2,652,214,153,610.37","2,559,910,411,068.77",-3.48


In [45]:
gini_change_09 = ((gini_2009 - gini_2014) / gini_2014) * 100
gini_change_09

-5.2853722007657584

In [46]:
elast_09 = tax_diffs_09['Pct. Change']['Combined'] / gini_change_09
elast_09

0.6584687151110622

### Combined Information

In [47]:
combo = pd.DataFrame({2000: [gini_2000, tax_diffs_00['Pct. Change']['Combined'],
                             gini_change_00, elast_00],
                      2004: [gini_2004, tax_diffs_04['Pct. Change']['Combined'],
                             gini_change_04, elast_04],
                      2009: [gini_2009, tax_diffs_09['Pct. Change']['Combined'],
                             gini_change_09, elast_09]})
combo.index = ['GINI', 'Pct. ∆ in Combined Liability', 'Pct ∆ in GINI', 'Elasticity']
combo

,2000,2004,2009
GINI,0.61,0.62,0.65
Pct. ∆ in Combined Liability,-4.94,-5.12,-3.48
Pct ∆ in GINI,-12.24,-10.00,-5.29
Elasticity,0.40,0.51,0.66


#### Combined Tax Liability Overtime

In [48]:
liability_00 = [combined_00]
liability_04 = [combined_04]
liability_09 = [combined_09]
liability_14 = [combined]
for year in range(2018, 2027):
    calc.advance_to_year(year)
    calc_2000.advance_to_year(year)
    calc_2004.advance_to_year(year)
    calc_2009.advance_to_year(year)
    calc.calc_all()
    calc_2000.calc_all()
    calc_2004.calc_all()
    calc_2009.calc_all()
    com_00 = (calc_2000.records.combined * calc_2000.records.s006).sum()
    liability_00.append(com_00)
    com_04 = (calc_2004.records.combined * calc_2004.records.s006).sum()
    liability_04.append(com_04)
    com_09 = (calc_2009.records.combined * calc_2009.records.s006).sum()
    liability_09.append(com_09)
    com_14 = (calc.records.combined * calc.records.s006).sum()
    liability_14.append(com_14)

In [49]:
yrs_list = [year for year in range(2017, 2027)]
liability_cds_00 = ColumnDataSource({'liability': [i * 1e-9 for i in liability_00],
                                     'year': yrs_list})
liability_cds_04 = ColumnDataSource({'liability': [i * 1e-9 for i in liability_04],
                                     'year': yrs_list})
liability_cds_09 = ColumnDataSource({'liability': [i * 1e-9 for i in liability_09],
                                     'year': yrs_list})
liability_cds_14 = ColumnDataSource({'liability': [i * 1e-9 for i in liability_14],
                                     'year': yrs_list})

In [50]:
f = figure(title='Combined Tax Liability by Distribution ($b)', width=800)
f.line(x='year', y='liability', line_width=2, legend='2000',
       source=liability_cds_00)
f.line(x='year', y='liability', line_width=2, color='red', legend='2004',
       source=liability_cds_04)
f.line(x='year', y='liability', line_width=2, color='green', legend='2009',
       source=liability_cds_09)
f.line(x='year', y='liability', line_width=2, color='orange', legend='2014',
       source=liability_cds_14)
f.xaxis.axis_label='Year'
f.yaxis.axis_label='Combined Tax Liability'
f.legend.location = 'top_left'
hover = HoverTool(tooltips=[('Year', '@year'), ('Liability', '@liability{$0.2f}')])
f.add_tools(hover)
f.xaxis[0].ticker.desired_num_ticks = 10
show(f)

## Sample Tax Reform Plans

In [51]:
# reduce income tax brackets
rate_reform = {
    2017: {
        '_II_rt1': [0.12],
        '_II_rt2': [0.12],
        '_II_rt3': [0.25],
        '_II_rt4': [0.25],
        '_II_rt5': [0.33],
        '_II_rt6': [0.33],
        '_II_rt7': [0.33]
    }
}
# Eliminate payroll tax cap
payroll_reform = {
    2017: {
        '_SS_Earnings_c': [250000]
    }
}

### Tax Rate Reform

In [52]:
# baseline calculator
calc14_tr = Calculator(records=Records(puf, gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                       weights=wt, adjust_ratios=adj, start_year=2011),
                       policy=Policy(gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                     start_year=2011, num_years=16),
                       consumption=Consumption(start_year=2011, num_years=16),
                       behavior=Behavior(start_year=2011, num_years=16))
calc14_tr.policy.implement_reform(rate_reform)

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


In [53]:
# 2000 distribution
calc00_tr = Calculator(records=Records(puf, gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                       weights=wt, adjust_ratios=adj_2000, start_year=2011),
                       policy=Policy(gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                     start_year=2011, num_years=16),
                       consumption=Consumption(start_year=2011, num_years=16),
                       behavior=Behavior(start_year=2011, num_years=16))
calc00_tr.policy.implement_reform(rate_reform)

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


In [54]:
# 2004 distribution
calc04_tr = Calculator(records=Records(puf, gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                       weights=wt, adjust_ratios=adj_2004, start_year=2011),
                       policy=Policy(gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                     start_year=2011, num_years=16),
                       consumption=Consumption(start_year=2011, num_years=16),
                       behavior=Behavior(start_year=2011, num_years=16))
calc04_tr.policy.implement_reform(rate_reform)

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


In [55]:
# 2009 distribution
calc09_tr = Calculator(records=Records(puf, gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                       weights=wt, adjust_ratios=adj_2009, start_year=2011),
                       policy=Policy(gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                     start_year=2011, num_years=16),
                       consumption=Consumption(start_year=2011, num_years=16),
                       behavior=Behavior(start_year=2011, num_years=16))
calc09_tr.policy.implement_reform(rate_reform)

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


In [56]:
delta_14 = []
delta_09 = []
delta_04 = []
delta_00 = []

In [57]:
# find liability effects over ten years
i = 0  # track index for liability lists
for year in range(2017, 2027):
    calc14_tr.advance_to_year(year)
    calc14_tr.calc_all()
    rev14 = (calc14_tr.records.combined * calc14_tr.records.s006).sum()
    delta_14.append(rev14 - liability_14[i])
    
    calc09_tr.advance_to_year(year)
    calc09_tr.calc_all()
    rev09 = (calc09_tr.records.combined * calc09_tr.records.s006).sum()
    delta_09.append(rev09 - liability_09[i])
    
    calc04_tr.advance_to_year(year)
    calc04_tr.calc_all()
    rev04 = (calc04_tr.records.combined * calc04_tr.records.s006).sum()
    delta_04.append(rev04 - liability_04[i])
    
    calc00_tr.advance_to_year(year)
    calc00_tr.calc_all()
    rev00 = (calc00_tr.records.combined * calc00_tr.records.s006).sum()
    delta_00.append(rev00 - liability_00[i])
    
    i += 1

In [58]:
# prep data for plotting
rev_cds14 = ColumnDataSource({'change': [i * 1e-9 for i in delta_14],
                              'year': yrs_list})
rev_cds09 = ColumnDataSource({'change': [i * 1e-9 for i in delta_09],
                              'year': yrs_list})
rev_cds04 = ColumnDataSource({'change': [i * 1e-9 for i in delta_04],
                              'year': yrs_list})
rev_cds00 = ColumnDataSource({'change': [i * 1e-9 for i in delta_00],
                              'year': yrs_list})

In [59]:
f = figure(title='Change in Tax Liability ($B)', width=750)
f.line(x='year', y='change', line_width=2, legend='2014', color='orange',
       source=rev_cds14)
f.line(x='year', y='change', line_width=2, legend='2009', color='green',
       source=rev_cds09)
f.line(x='year', y='change', line_width=2, legend='2004', color='red',
       source=rev_cds04)
f.line(x='year', y='change', line_width=2, legend='2000', color='blue',
       source=rev_cds00)
f.xaxis[0].ticker.desired_num_ticks = 10
f.xaxis.axis_label = 'Year'
f.yaxis.axis_label = 'Change in Tax Liability'
hover = HoverTool(tooltips=[('Year', '@year'), ('Change', '@change{$0.2f}')])
f.add_tools(hover)
show(f)

### Payroll Tax Reform

In [60]:
# baseline calculator
calc14_pr = Calculator(records=Records(puf, gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                       weights=wt, adjust_ratios=adj, start_year=2011),
                       policy=Policy(gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                     start_year=2011, num_years=16),
                       consumption=Consumption(start_year=2011, num_years=16),
                       behavior=Behavior(start_year=2011, num_years=16))
calc14_pr.policy.implement_reform(payroll_reform)

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


In [61]:
# 2000 distribution
calc00_pr = Calculator(records=Records(puf, gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                       weights=wt, adjust_ratios=adj_2000, start_year=2011),
                       policy=Policy(gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                     start_year=2011, num_years=16),
                       consumption=Consumption(start_year=2011, num_years=16),
                       behavior=Behavior(start_year=2011, num_years=16))
calc00_pr.policy.implement_reform(payroll_reform)

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


In [62]:
# 2004 distribution
calc04_pr = Calculator(records=Records(puf, gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                       weights=wt, adjust_ratios=adj_2004, start_year=2011),
                       policy=Policy(gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                     start_year=2011, num_years=16),
                       consumption=Consumption(start_year=2011, num_years=16),
                       behavior=Behavior(start_year=2011, num_years=16))
calc04_pr.policy.implement_reform(payroll_reform)

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


In [63]:
# 2009 distribution
calc09_pr = Calculator(records=Records(puf, gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                       weights=wt, adjust_ratios=adj_2009, start_year=2011),
                       policy=Policy(gfactors=Growfactors('../taxdata/stage1/growfactors.csv'),
                                     start_year=2011, num_years=16),
                       consumption=Consumption(start_year=2011, num_years=16),
                       behavior=Behavior(start_year=2011, num_years=16))
calc09_pr.policy.implement_reform(payroll_reform)

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


In [64]:
delta_14pr = []
delta_09pr = []
delta_04pr = []
delta_00pr = []

In [65]:
# find liability effects over ten years
i = 0  # track index for liability lists
for year in range(2017, 2027):
    calc14_pr.advance_to_year(year)
    calc14_pr.calc_all()
    rev14 = (calc14_pr.records.combined * calc14_pr.records.s006).sum()
    delta_14pr.append(rev14 - liability_14[i])
    
    calc09_pr.advance_to_year(year)
    calc09_pr.calc_all()
    rev09 = (calc09_pr.records.combined * calc09_pr.records.s006).sum()
    delta_09pr.append(rev09 - liability_09[i])
    
    calc04_pr.advance_to_year(year)
    calc04_pr.calc_all()
    rev04 = (calc04_pr.records.combined * calc04_pr.records.s006).sum()
    delta_04pr.append(rev04 - liability_04[i])
    
    calc00_pr.advance_to_year(year)
    calc00_pr.calc_all()
    rev00 = (calc00_pr.records.combined * calc00_pr.records.s006).sum()
    delta_00pr.append(rev00 - liability_00[i])
    
    i += 1

In [66]:
# prep data for plotting
rev_cds14pr = ColumnDataSource({'change': [i * 1e-9 for i in delta_14pr],
                                'year': yrs_list})
rev_cds09pr = ColumnDataSource({'change': [i * 1e-9 for i in delta_09pr],
                                'year': yrs_list})
rev_cds04pr = ColumnDataSource({'change': [i * 1e-9 for i in delta_04pr],
                                'year': yrs_list})
rev_cds00pr = ColumnDataSource({'change': [i * 1e-9 for i in delta_00pr],
                                'year': yrs_list})

In [67]:
f = figure(title='Change in Tax Liability ($B)', width=750)
f.line(x='year', y='change', line_width=2, legend='2014', color='orange',
       source=rev_cds14pr)
f.line(x='year', y='change', line_width=2, legend='2009', color='green',
       source=rev_cds09pr)
f.line(x='year', y='change', line_width=2, legend='2004', color='red',
       source=rev_cds04pr)
f.line(x='year', y='change', line_width=2, legend='2000', color='blue',
       source=rev_cds00pr)
f.xaxis[0].ticker.desired_num_ticks = 10
f.xaxis.axis_label = 'Year'
f.yaxis.axis_label = 'Change in Tax Liability'
hover = HoverTool(tooltips=[('Year', '@year'), ('Change', '@change{$0.2f}')])
f.add_tools(hover)
f.legend.location='top_left'
show(f)

There really are four lines in the graph above. The liability change is just almost identical for each calculator